In [9]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%autoreload 2

import sys
import os

sys.path.append('../')

from utilities import create_data as create
from autogluon.tabular import TabularDataset, TabularPredictor

home_dir = os.path.join(os.path.expanduser('~'), 'Documents')
saved_data_path = os.path.join(home_dir, 'TDT4173-ML/data_results')
saved_model_path = os.path.join(home_dir, 'models/AutoGluonModels')
load_data_path = os.path.join(home_dir, 'TDT4173-ML/data_feature_engineering')

In [11]:
import pandas as pd

train = pd.read_csv(os.path.join(load_data_path, 'no_duplicates_train.csv'))
test = pd.read_csv(os.path.join(load_data_path, 'no_duplicates_test.csv'))

AutoGluon parameters

In [14]:
label = 'pv_measurement'
metric = 'mean_absolute_error'
path = os.path.join(saved_model_path, '08E_no_duplicates_best_quality')

In [15]:
predictor = TabularPredictor(label='pv_measurement',
                            #  problem_type='regression',
                             eval_metric=metric,
                             path=path,
                            #  verbosity=3,
                            #  sample_weight=None,
                            #  weight_evaluation=False
                             )

predictor.fit(train_data=train,
            #   presets='best_quality'
              )

Beginning AutoGluon training ...
AutoGluon will save models to "/home/stinky/Documents/models/AutoGluonModels/08E_no_duplicates_best_quality/"
AutoGluon Version:  0.8.2
Python Version:     3.10.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #37~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Mon Oct  9 15:34:04 UTC 2
Disk Space Avail:   168.39 GB / 250.38 GB (67.3%)
Train Data Rows:    61623
Train Data Columns: 51
Label Column: pv_measurement
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (5733.42, -0.0, 292.84546, 871.53848)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	A

KeyboardInterrupt: 

In [ ]:
predictor.leaderboard()

In [ ]:
predictor.fit_summary(show_plot=True)

In [ ]:
predictor.feature_importance(train_data=train)

In [ ]:
best_model = predictor.get_model_best()
print(best_model)

In [29]:
test = TabularDataset(test)

y_pred = predictor.predict(test)

date_forecast
2023-05-01 00:00:00     -1.510917
2023-05-01 01:00:00     -0.930792
2023-05-01 02:00:00     -1.094912
2023-05-01 03:00:00     44.515774
2023-05-01 04:00:00    253.171509
Name: pv_measurement, dtype: float32

In [38]:
import pandas as pd

predictions = pd.DataFrame(y_pred)
predictions.reset_index(inplace=True)
predictions.drop(columns=['date_forecast'], inplace=True)

# Set all negative values to zero
predictions[predictions < 0] = 0

predictions.to_csv(os.path.join(saved_data_path, '08E_autogluon_no_duplicates.csv'), index=True, index_label='id', header=['predictions'])